In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import plotly.express as px
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

from utils.paths import BOOKS_PATH


/home/mateusz/dev/lubimyczytac_project/lubimyczytac/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


# ======= 1. Wczytanie danych =======
df_books = pd.read_csv(BOOKS_PATH)  # Zmień ścieżkę na swoją

# ======= 2. Tworzenie etykiety bestsellerów (≥ 2000 posiadaczy) =======
bestseller_threshold = 2000
df_books["bestseller"] = (df_books["number_of_people_has"] >= bestseller_threshold).astype(int)

# ======= 3. Feature Engineering =======
df_books["description_length"] = df_books["description"].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)

rating_features = [f"rating_{i}" for i in range(1, 11)]
top_authors = df_books["author"].value_counts().index[:100]
df_books["is_top_author"] = df_books["author"].apply(lambda x: 1 if x in top_authors else 0)

categorical_features = ["Kategoria", "Format", "Cykl"]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in categorical_features:
    df_books[col] = df_books[col].fillna("Unknown")
    df_books[col] = le.fit_transform(df_books[col])

# ======= 4. Wybór finalnych cech =======
selected_features = ["description_length", "is_top_author"] + rating_features + categorical_features
df_class_final = df_books[selected_features + ["bestseller"]].dropna()

# ======= 5. UMAP - Redukcja wymiarów do 3D =======
X_umap = df_class_final[selected_features]
y_umap = df_class_final["bestseller"]

scaler_umap = StandardScaler()
X_umap_scaled = scaler_umap.fit_transform(X_umap)

# Wykonanie UMAP
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=3, random_state=69420)
X_umap_3d = reducer.fit_transform(X_umap_scaled)

# Tworzenie DataFrame z wynikami UMAP
df_umap = pd.DataFrame(X_umap_3d, columns=["UMAP1", "UMAP2", "UMAP3"])
df_umap["bestseller"] = y_umap.values

# ======= 6. Wizualizacja interaktywna 3D w Plotly =======
fig = px.scatter_3d(df_umap, x="UMAP1", y="UMAP2", z="UMAP3", 
                     color=df_umap["bestseller"].astype(str),
                     title="UMAP 3D - Bestseller vs Nie-Bestseller",
                     labels={"color": "Bestseller"},
                     opacity=0.7)
fig.update_layout(width=800, height=600)
fig.show()


/tmp/ipykernel_283142/3553194098.py:2: DtypeWarning: Columns (76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv(BOOKS_PATH)  # Zmień ścieżkę na swoją
/home/mateusz/dev/lubimyczytac_project/lubimyczytac/lib64/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/mateusz/dev/lubimyczytac_project/lubimyczytac/lib64/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [5]:
df_umap.to_csv("data/umap_3d_bestseller.csv", index=False)

In [6]:
# group df umap in clusters
from sklearn.cluster import KMeans

In [7]:
# find number of clusters with elbow method
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=80082)
    kmeans.fit(X_umap_3d)
    inertia.append(kmeans.inertia_)

px.line(x=range(1, 11), y=inertia, title="Elbow Method - UMAP 3D")

In [8]:
kmeans = KMeans(n_clusters=5, random_state=80082)
kmeans.fit(X_umap_3d)

KMeans(n_clusters=5, random_state=80082)

In [9]:
import plotly.express as px


df_umap["cluster"] = kmeans.labels_
# fig = px.scatter_3d(df_umap, x="UMAP1", y="UMAP2", z="UMAP3", color="cluster", opacity=0.7,
#                     color_discrete_sequence=dark_colors)  # Use a color sequence suitable for white background
fig = px.scatter_3d(df_umap, x="UMAP1", y="UMAP2", z="UMAP3", 
                     color=df_umap["cluster"].astype(str),
                     title="Groups Identified by KMeans",
                     labels={"color": "Bestseller"},
                     opacity=0.7)

fig.update_layout(width=800, height=600, legend_title_text='Cluster')  # Set the figure size and legend title
fig.show()

In [11]:
df_umap.drop('bestseller', axis=1).to_csv('data/umap_3d_clusters.csv', index=False)